In [1]:
import AmazonReviews

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
ar = AmazonReviews.AmazonReviews()

PATH = '../data/amazon_reviews_us_Toys_v1_00.tsv'
ar.load_data(PATH)

In [69]:
ar.calc_trend_score(review_days=30)

Review the distribution of the first review data. 1/1/2014 is the most popular. May need to move the cutoff date.

2014-01-01    0.030230
2014-01-02    0.029510
2014-01-03    0.026328
2014-01-04    0.016026
2014-01-07    0.014318

In [70]:
ar.reviews_selected_df.min_review_date.value_counts(normalize=True).sort_values(ascending = False).head()

2014-01-01    0.030230
2014-01-02    0.029510
2014-01-03    0.026328
2014-01-04    0.016026
2014-01-07    0.014318
Name: min_review_date, dtype: float64

In [71]:
ar.product_trend_df[ar.product_trend_df.trend == 1].describe()

,count,median,std,orig_std,review_success,trend_score,trend
count,5469.000000,5469.000000,5469.000000,5469.000000,5469.000000,5469.000000,5469.0
mean,73.554032,10.885160,2.182931,2.114140,11.285563,0.999998,1.0
std,81.650357,0.954892,1.027999,1.149140,9.822584,0.000003,0.0
min,8.000000,5.196152,0.463900,0.000000,5.964506,0.999987,1.0
25%,30.000000,11.180340,1.386188,1.386188,6.869424,0.999998,1.0
50%,55.000000,11.180340,2.382850,2.382850,8.231919,1.000000,1.0
75%,89.000000,11.180340,2.968471,2.968471,11.643701,1.000000,1.0
max,1268.000000,11.180340,4.365866,4.365866,167.967345,1.000000,1.0


In [72]:
ar.create_observations()

In [73]:
ar.create_train_test_split()

## DTM Creation

Create DTM and only restrict features to English words in the `nltk.corpus.words`

In [97]:
from nltk.corpus import words

words_corpus = set(words.words())
analyzer = CountVectorizer().build_analyzer()

def english_corpus(doc):
    return [w for w in analyzer(doc) if w in words_corpus]


ar.add_dtm(CountVectorizer(stop_words='english', analyzer=english_corpus, min_df=0.005),'count_1_gram')
# ar.add_dtm(CountVectorizer(stop_words='english', min_df=0.005),'count_1_gram')

features = ar.X_train['count_1_gram'][1]

len(features)

485

* 388 words when not using the english_corpus
* 485 words when using the english_corpus --- somthing seems off

In [92]:
features

['about',
 'absolutely',
 'actually',
 'adorable',
 'after',
 'again',
 'all',
 'almost',
 'also',
 'always',
 'am',
 'an',
 'and',
 'another',
 'any',
 'are',
 'around',
 'as',
 'at',
 'awesome',
 'back',
 'bad',
 'be',
 'beautiful',
 'because',
 'been',
 'before',
 'being',
 'belt',
 'best',
 'better',
 'big',
 'bigger',
 'bit',
 'black',
 'body',
 'bottom',
 'bought',
 'bra',
 'brand',
 'but',
 'buy',
 'by',
 'came',
 'can',
 'cheap',
 'coat',
 'color',
 'colors',
 'come',
 'comfortable',
 'comfy',
 'cool',
 'costume',
 'cotton',
 'could',
 'cut',
 'cute',
 'daughter',
 'day',
 'days',
 'definitely',
 'design',
 'did',
 'different',
 'disappointed',
 'do',
 'does',
 'don',
 'down',
 'dress',
 'easy',
 'enough',
 'even',
 'ever',
 'every',
 'everything',
 'exactly',
 'excellent',
 'extra',
 'fabric',
 'far',
 'fast',
 'favorite',
 'feel',
 'few',
 'find',
 'fine',
 'first',
 'fit',
 'for',
 'found',
 'from',
 'front',
 'get',
 'gift',
 'give',
 'go',
 'going',
 'good',
 'got',
 'grea

In [80]:
X_train = ar.X_train['count_1_gram'][0]

KeyboardInterrupt: 

In [32]:
ar.reviews_selected_df.marketplace.unique()

array(['US'], dtype=object)

In [35]:
len(words.words())

236736

In [36]:
len(features)

93756

In [38]:
words_corpus = set(words.words())
feature_english = [f for f in features if f in words_corpus]

In [39]:
len(feature_english)

24165

In [40]:
ar.obs.shape

(1781440, 5)

In [41]:
len(words_corpus)

235892

In [43]:
feature_english

['zoom',
 'zoo',
 'zoned',
 'zone',
 'zonal',
 'zombie',
 'zoll',
 'zoid',
 'zoic',
 'zodiac',
 'zo',
 'zirconium',
 'zirconia',
 'zippy',
 'zipping',
 'zipper',
 'zip',
 'zing',
 'zinfandel',
 'zinc',
 'zigzagged',
 'zigzag',
 'zig',
 'zeta',
 'zesty',
 'zest',
 'zero',
 'zeppelin',
 'zephyr',
 'zenana',
 'zee',
 'zed',
 'zebra',
 'zealous',
 'zeal',
 'zany',
 'zander',
 'zain',
 'zagged',
 'zag',
 'zac',
 'za',
 'yus',
 'yurt',
 'yummy',
 'yuletide',
 'yule',
 'yuh',
 'yucky',
 'yuck',
 'yuan',
 'yr',
 'yoy',
 'yow',
 'youve',
 'youthful',
 'youth',
 'youse',
 'youngster',
 'youngish',
 'younger',
 'young',
 'youd',
 'yorker',
 'york',
 'yore',
 'yor',
 'yonder',
 'yond',
 'yon',
 'yolked',
 'yolk',
 'yoke',
 'yok',
 'yoi',
 'yogi',
 'yoga',
 'yodel',
 'yo',
 'ym',
 'yin',
 'yielding',
 'yield',
 'yesteryear',
 'yesterday',
 'yester',
 'yest',
 'yes',
 'yer',
 'yep',
 'yeoman',
 'yen',
 'yelp',
 'yellowy',
 'yellows',
 'yellowly',
 'yellowish',
 'yellowing',
 'yellow',
 'yelling',
 '